In [1]:
# Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import numpy as np
import time 
import json

In [2]:
#Scrape the Season Schedule
full_schedule = []
year_url = "https://www.footballdb.com/games/index.html?lg=NFL&yr=2017"
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless = True)
browser.visit(year_url)
year_html = browser.html
year_soup = BeautifulSoup(year_html, 'html.parser')
print ("--- Chrome Driver Complete ---")
#create schedule of full season with date and URL for Box Score

tables = year_soup.findAll("table", class_="statistics")
for x in np.arange(len(tables)):
    table = tables[x].findAll("tbody")
    for t in table:
        rows = t.findAll("tr")
        for row in rows:
            d = row.findAll("td")
            date = d[0].text[:10]
            for a in row.findAll("a", href = True):
                gameURL = a["href"]
            full_schedule.append({"date":date, "gameURL": gameURL})
#             print (f'{date} // {gameURL} URL acquired')
# print (full_schedule)
print (f"--- Schedule Scraped ---")
    

--- Chrome Driver Complete ---
--- Schedule Scraped ---


In [3]:
print (full_schedule)

[{'date': '09/07/2017', 'gameURL': '/games/boxscore.html?gid=2017090701'}, {'date': '09/10/2017', 'gameURL': '/games/boxscore.html?gid=2017091001'}, {'date': '09/10/2017', 'gameURL': '/games/boxscore.html?gid=2017091002'}, {'date': '09/10/2017', 'gameURL': '/games/boxscore.html?gid=2017091003'}, {'date': '09/10/2017', 'gameURL': '/games/boxscore.html?gid=2017091004'}, {'date': '09/10/2017', 'gameURL': '/games/boxscore.html?gid=2017091006'}, {'date': '09/10/2017', 'gameURL': '/games/boxscore.html?gid=2017091009'}, {'date': '09/10/2017', 'gameURL': '/games/boxscore.html?gid=2017091012'}, {'date': '09/10/2017', 'gameURL': '/games/boxscore.html?gid=2017091013'}, {'date': '09/10/2017', 'gameURL': '/games/boxscore.html?gid=2017091010'}, {'date': '09/10/2017', 'gameURL': '/games/boxscore.html?gid=2017091005'}, {'date': '09/10/2017', 'gameURL': '/games/boxscore.html?gid=2017091008'}, {'date': '09/10/2017', 'gameURL': '/games/boxscore.html?gid=2017091007'}, {'date': '09/11/2017', 'gameURL': '/g

In [23]:
#Iterate through URLS
stats_db = []
for game in full_schedule:
    while True:
        try:
        #     print(game)
            gameday = game['date']
            url_ending = game['gameURL']
            game_url = "https://www.footballdb.com" + url_ending
            executable_path = {'executable_path': 'chromedriver.exe'}
            browser = Browser('chrome', **executable_path, headless = True)
            browser.visit(game_url)
            game_html = browser.html
            game_soup = BeautifulSoup(game_html, 'html.parser')
            left_table = game_soup.findAll("div", class_="section_left")[0].findAll("tr")
            right_table = game_soup.findAll("div", class_="section_right")[0].findAll("tr")
            score_table = game_soup.findAll("table", class_="statistics")[0].findAll("tr")
            print ("--- Game URL Scraping ---")

            # Create Dictionaries for game by team

            away_data = []
            home_data = []
            for row in left_table:
                tableData = row.findAll("td")
                away_data.append(tableData[1].text)
                home_data.append(tableData[2].text)
            for row in right_table[1:]:
                tableData = row.findAll("td")
                away_data.append(tableData[1].text)
                home_data.append(tableData[2].text)
            try:
                awayScore = int(score_table[1].findAll("td")[6].text)
                homeScore = int(score_table[2].findAll("td")[6].text)
            except:
                awayScore = int(score_table[1].findAll("td")[5].text)
                homeScore = int(score_table[2].findAll("td")[5].text)
            totalScore = awayScore + homeScore
            awayMargin = awayScore - homeScore
            homeMargin = homeScore - awayScore
            if homeMargin < 0:
                awayResult = "Win"
                homeResult = "Loss"
            elif homeMargin == 0:
                awayResult = "Tie"
                homeResult = "Tie"
            elif homeMargin > 0:
                awayResult = "Loss"
                homeResult = "Win"

        # Create Stats DataBase


            away_stats = {"date": gameday,
                "team": away_data[0],
                "points": awayScore,
                "opp": home_data[0],
                "ha": "away",
                "points_allowed": homeScore,
                "total_points": totalScore,
                "margin": awayMargin,
                "result": awayResult,
                "first_downs": int(away_data[1]),
                "total_yards": int(away_data[5]),
                "rush_yards": int(away_data[6]),
                "pass_yards": int(away_data[9]),
                "penalty_yards": int(away_data[19].split("-")[1]),
                "sacks": int(home_data[11].split("-")[0]),
                "takeaways": int(home_data[10].split("-")[2])+int(home_data[20].split("-")[1]),
                "3d%": float(away_data[22].split("-")[0])/float(away_data[22].split("-")[1]),
                "plays": int(away_data[24]),
                "TOP": float(away_data[26].split(":")[0])+(float(away_data[26].split(":")[1])/60),
                "first_downs_allowed": int(home_data[1]),
                "total_yards_allowed": int(home_data[5]),
                "rush_yards_allowed": int(home_data[6]),
                "pass_yards_allowed": int(home_data[9]),
                "sacked": int(away_data[11].split("-")[0]),
                "turnovers": int(away_data[10].split("-")[2])+int(away_data[20].split("-")[1]),
                "3d%_allowed": float(home_data[22].split("-")[0])/float(home_data[22].split("-")[1])
            }
        #     print(away_stats)
            home_stats = {"date": gameday,
                "team": home_data[0],
                "points": homeScore,
                "opp": away_data[0],
                "ha": "home",
                "points_allowed": awayScore,
                "total_points": totalScore,
                "margin": homeMargin,
                "result": homeResult,
                "first_downs": int(home_data[1]),
                "total_yards": int(home_data[5]),
                "rush_yards": int(home_data[6]),
                "pass_yards": int(home_data[9]),
                "penalty_yards": int(home_data[19].split("-")[1]),
                "sacks": int(away_data[11].split("-")[0]),
                "takeaways": int(away_data[10].split("-")[2])+int(away_data[20].split("-")[1]),
                "3d%": float(home_data[22].split("-")[0])/float(home_data[22].split("-")[1]),
                "plays": int(home_data[24]),
                "TOP": float(home_data[26].split(":")[0])+(float(home_data[26].split(":")[1])/60),
                "first_downs_allowed": int(away_data[1]),
                "total_yards_allowed": int(away_data[5]),
                "rush_yards_allowed": int(away_data[6]),
                "pass_yards_allowed": int(away_data[9]),
                "sacked": int(home_data[11].split("-")[0]),
                "turnovers": int(home_data[10].split("-")[2])+int(home_data[20].split("-")[1]),
                "3d%_allowed": float(away_data[22].split("-")[0])/float(away_data[22].split("-")[1])
            }
        #     print(home_stats)
            stats_db.append(away_stats)
            stats_db.append(home_stats)
            print (f'{away_stats["team"]} vs. {home_stats["team"]} on {gameday} complete!')
            break
        except:
            pass

# print (stats_db)
print ("--- Stats Scraped ---")

--- Game URL Scraping ---
TennesseeTen vs. PittsburghPit on 09/10/2009 complete!
--- Game URL Scraping ---
MiamiMia vs. AtlantaAtl on 09/13/2009 complete!
--- Game URL Scraping ---
DenverDen vs. CincinnatiCin on 09/13/2009 complete!
--- Game URL Scraping ---
MinnesotaMin vs. ClevelandCle on 09/13/2009 complete!
--- Game URL Scraping ---
JacksonvilleJax vs. IndianapolisInd on 09/13/2009 complete!
--- Game URL Scraping ---
DetroitDet vs. New OrleansNO on 09/13/2009 complete!
--- Game URL Scraping ---
DallasDal vs. Tampa BayTB on 09/13/2009 complete!
--- Game URL Scraping ---
PhiladelphiaPhi vs. CarolinaCar on 09/13/2009 complete!
--- Game URL Scraping ---
Kansas CityKC vs. BaltimoreBal on 09/13/2009 complete!
--- Game URL Scraping ---
NY JetsNYJ vs. HoustonHou on 09/13/2009 complete!
--- Game URL Scraping ---
WashingtonWas vs. NY GiantsNYG on 09/13/2009 complete!
--- Game URL Scraping ---
San FranciscoSF vs. ArizonaAri on 09/13/2009 complete!
--- Game URL Scraping ---
St. LouisStl vs. Se

--- Game URL Scraping ---
ClevelandCle vs. ChicagoChi on 11/01/2009 complete!
--- Game URL Scraping ---
SeattleSea vs. DallasDal on 11/01/2009 complete!
--- Game URL Scraping ---
St. LouisStl vs. DetroitDet on 11/01/2009 complete!
--- Game URL Scraping ---
San FranciscoSF vs. IndianapolisInd on 11/01/2009 complete!
--- Game URL Scraping ---
MiamiMia vs. NY JetsNYJ on 11/01/2009 complete!
--- Game URL Scraping ---
DenverDen vs. BaltimoreBal on 11/01/2009 complete!
--- Game URL Scraping ---
NY GiantsNYG vs. PhiladelphiaPhi on 11/01/2009 complete!
--- Game URL Scraping ---
JacksonvilleJax vs. TennesseeTen on 11/01/2009 complete!
--- Game URL Scraping ---
OaklandOak vs. San DiegoSD on 11/01/2009 complete!
--- Game URL Scraping ---
MinnesotaMin vs. Green BayGB on 11/01/2009 complete!
--- Game URL Scraping ---
CarolinaCar vs. ArizonaAri on 11/01/2009 complete!
--- Game URL Scraping ---
AtlantaAtl vs. New OrleansNO on 11/02/2009 complete!
--- Game URL Scraping ---
WashingtonWas vs. AtlantaAtl

--- Game URL Scraping ---
IndianapolisInd vs. JacksonvilleJax on 12/17/2009 complete!
--- Game URL Scraping ---
DallasDal vs. New OrleansNO on 12/19/2009 complete!
--- Game URL Scraping ---
New EnglandNE vs. BuffaloBuf on 12/20/2009 complete!
--- Game URL Scraping ---
ArizonaAri vs. DetroitDet on 12/20/2009 complete!
--- Game URL Scraping ---
MiamiMia vs. TennesseeTen on 12/20/2009 complete!
--- Game URL Scraping ---
ClevelandCle vs. Kansas CityKC on 12/20/2009 complete!
--- Game URL Scraping ---
HoustonHou vs. St. LouisStl on 12/20/2009 complete!
--- Game URL Scraping ---
AtlantaAtl vs. NY JetsNYJ on 12/20/2009 complete!
--- Game URL Scraping ---
CincinnatiCin vs. San DiegoSD on 12/20/2009 complete!
--- Game URL Scraping ---
OaklandOak vs. DenverDen on 12/20/2009 complete!
--- Game URL Scraping ---
San FranciscoSF vs. PhiladelphiaPhi on 12/20/2009 complete!
--- Game URL Scraping ---
Green BayGB vs. PittsburghPit on 12/20/2009 complete!
--- Game URL Scraping ---
ChicagoChi vs. Baltimor

In [24]:
nfl = pd.DataFrame(stats_db)
nfl.head()

,3d%,3d%_allowed,TOP,date,first_downs,first_downs_allowed,ha,margin,opp,pass_yards,...,rush_yards,rush_yards_allowed,sacked,sacks,takeaways,team,total_points,total_yards,total_yards_allowed,turnovers
0,0.307692,0.285714,28.350000,09/10/2009,18,19,away,-3,PittsburghPit,234,...,86,36,1,4,3,TennesseeTen,3,320,357,2
1,0.285714,0.307692,36.183333,09/10/2009,19,18,home,3,TennesseeTen,321,...,36,86,4,1,2,PittsburghPit,3,357,320,3
2,0.363636,0.400000,29.116667,09/13/2009,16,19,away,-12,AtlantaAtl,163,...,96,68,4,2,0,MiamiMia,26,259,281,4
3,0.400000,0.363636,30.883333,09/13/2009,19,16,home,12,MiamiMia,213,...,68,96,2,4,4,AtlantaAtl,26,281,259,0
4,0.250000,0.333333,26.550000,09/13/2009,10,16,away,5,CincinnatiCin,227,...,75,86,3,3,2,DenverDen,19,302,307,0


In [25]:
nfl.to_csv("nfl_2017.csv")